In [5]:
%pip install tabula
import pandas as pd
import tabula
import numpy as np
from IPython.utils import io
import matplotlib.pyplot as plt
import plotly.graph_objects as go

Note: you may need to restart the kernel to use updated packages.


In [3]:
try:
    with io.capture_output():
        %run -i "Informations lieux des compétitions.ipynb"
except Exception as e:
    display(e)

In [15]:
# Suppression des tirets et mise en majuscule du début de chaque mot dans la colonne "Lieu"
formatted_lieux = df['Lieu'].apply(lambda x: x.replace('-', ' ').title())

# Création du diagramme à barres interactif avec Plotly en utilisant les nouvelles étiquettes
fig = go.Figure(data=[
    go.Bar(x=formatted_lieux, y=df["Nombre de personnes à l'événément"])
])

# Mise à jour du titre et des étiquettes
fig.update_layout(
    title="Nombre de personnes attendues aux lieux de compétitions",
    xaxis=dict(title='Lieu'),
    yaxis=dict(title="Nombre de personnes"),
    xaxis_tickangle=-45  # Rotation des étiquettes x pour une meilleure lisibilité
)

# Affichage du graphique
fig.show()
